In [25]:
%matplotlib widget
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment, clean_exp_with_hyphaes
from hyphae_id import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    solve_degree4,
    clean_obvious_fake_tips,
)
from extract_graph import prune_graph

In [4]:
plate = 13
exp_clean = pickle.load(open(f"Data/" + f"experiment_{plate}.pick", "rb"))

In [ ]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)

In [6]:
solved = solve_degree4(exp_clean)

[5576, 1580, 6170, 6171, 5151]
6 5150 [(Edge(5150,5152), Edge(5150,6172)), (Edge(5150,6171), Edge(5150,5151))] 2
[5576, 1580, 6170, 6171, 5151]
7 5150 [(Edge(5150,6569), Edge(5150,6172)), (Edge(5150,6171), Edge(5150,5151))] 2


In [26]:
clean_obvious_fake_tips(exp_clean)

In [27]:
hyphs, gr_inf = save_hyphaes(exp_clean)
exp_clean.save()

2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
2303 2303
3801 3801
3801 3801
3801 3801
3801 3801
3801 3801
3801 3801
3801 3801
3801 3801
4002 4002
4002 4002
4002 4002
4002 4002
4002 4002
4002 4002
4002 4002
4002 4002
4002 4002
4500 4500
4500 4500
4500 4500
4500 4500
4998 4998
4998 4998
4998 4998
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
7208 7208
8352 8352
8449 8449
8449 8449
8449 8449
8449 8449
8450 8450
8473 8473
8473 8473
8473 8473
10339 10339
10641 10641
10783 10783
10783 10783
10783 10783
10783 10783
10783 10783
10783 10783
11889 11889
11889 11889
11889 11889
11889 11889
11889 11889
12425 12425
12425 12425
12425 12425
12425 12425
12428 12428
12428 12428
12428 12428


In [15]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()

In [144]:
hyph_anas_tip_tip = []
potential = []
for hyph in exp_clean.hyphaes:
    if (
        len(hyph.ts) >= 2
        and hyph.end.degree(hyph.ts[-1]) == 1
        and hyph.end.ts()[-1] != len(exp_clean.nx_graph) - 1
        and not np.all([hyph.get_length_pixel(t) <= 20 for t in hyph.ts])
    ):
        potential.append(hyph)
for hyph in potential:
    t0 = hyph.ts[-1]
    for hyph2 in potential:
        if hyph2.ts[-1] == t0 and hyph != hyph2:
            vector = (hyph2.end.pos(t0) - hyph.end.pos(t0)) / np.linalg.norm(
                hyph2.end.pos(t0) - hyph.end.pos(t0)
            )
            vertical_vector = np.array([-1, 0])
            dot_product = np.dot(vertical_vector, vector)
            if (
                vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
            ):  # determinant
                angle = np.arccos(dot_product) / (2 * np.pi) * 360
            else:
                angle = -np.arccos(dot_product) / (2 * np.pi) * 360
            score = np.cos(
                (angle - (180 + hyph.end.edges(t0)[0].orientation_begin(t0, 30)))
                / 360
                * 2
                * np.pi
            ) + np.cos(
                (360 + angle - hyph2.end.edges(t0)[0].orientation_begin(t0, 30))
                / 360
                * 2
                * np.pi
            )
            if (
                np.linalg.norm(hyph2.end.pos(t0) - hyph.end.pos(t0)) <= 500
                and score >= 0.5
            ):
                hyph_anas_tip_tip.append((hyph, hyph2, t0))

In [145]:
hyph_anas_tip_hyph = [
    hyphat
    for hyphat in exp_clean.hyphaes
    if len(hyphat.ts) >= 1
    and hyphat.end.degree(hyphat.ts[-1]) >= 3
    and hyphat.end.degree(hyphat.ts[-2]) >= 3
]

In [146]:
len(hyph_anas_tip_hyph)

239

In [136]:
hyph_anas_tip_tip

[(Hyphae(126,454), Hyphae(4401,4398), 1.954672336101087),
 (Hyphae(161,179), Hyphae(6267,6266), 0.7809787080345737),
 (Hyphae(439,442), Hyphae(464,495), 1.8130618045327542),
 (Hyphae(464,495), Hyphae(439,442), 1.8130618045327547),
 (Hyphae(813,773), Hyphae(845,721), 1.5834953873197146),
 (Hyphae(845,721), Hyphae(813,773), 1.583495387319714),
 (Hyphae(877,1029), Hyphae(6980,6979), 1.7938215538705315),
 (Hyphae(929,909), Hyphae(1204,1293), 1.7983931480030908),
 (Hyphae(1038,1051), Hyphae(2693,2684), 1.7180008032863814),
 (Hyphae(1204,1293), Hyphae(929,909), 1.798393148003091),
 (Hyphae(1603,1694), Hyphae(3932,3929), 1.909004120406454),
 (Hyphae(2088,2127), Hyphae(3194,3195), 1.047640346299485),
 (Hyphae(2693,2684), Hyphae(1038,1051), 1.7180008032863805),
 (Hyphae(3194,3195), Hyphae(2088,2127), 1.0476403462994843),
 (Hyphae(3932,3929), Hyphae(1603,1694), 1.9090041204064536),
 (Hyphae(4056,4060), Hyphae(5608,5604), 1.9795207132807826),
 (Hyphae(4401,4398), Hyphae(126,454), 1.95467233610108

In [140]:
anas1, anas2, score = choice(hyph_anas_tip_tip)
t0 = anas1.ts[-1]
plt.close("all")

In [141]:
anas1, anas1.ts, anas2, anas2.ts, score

(Hyphae(2693,2684),
 [1, 2, 3],
 Hyphae(1038,1051),
 [0, 1, 2, 3],
 1.7180008032863805)

In [142]:
exp_clean.plot(
    [t0, t0 + 1],
    [[anas1.end.label, anas1.root.label, anas2.end.label, anas2.root.label]] * 2,
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
len(solved)

500

In [19]:
case = choice(solved)
t0 = case[0]
nodes = case[1]
plt.close("all")

In [14]:
len(solved)

500

In [17]:
t0

14

In [20]:
exp_clean.plot([9, t0], [[node.label for node in nodes]] * 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
exp_clean.plot([0, 10, 20], [[118, 144, 146, 144, 134, 144, 155]] * 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …